# bioimageio.core usage examples

Checkout [load_model_and_create_your_own.ipynb](https://github.com/bioimage-io/spec-bioimage-io/blob/main/example/) for examples on model creation, loading and inspection.

In [ ]:
import os

if os.getenv("COLAB_RELEASE_TAG"):
    %pip install bioimageio.core torch onnxruntime

In [ ]:
# helper function for showing multiple images in napari
from bioimageio.core import Tensor
from bioimageio.spec.utils import download, load_array
from typing import Any, Dict, Union
from pathlib import Path

import matplotlib.pyplot as plt
from numpy.typing import NDArray

try:
    import napari
except ImportError:

    def show_images(images: Dict[str, Union[Tensor, NDArray[Any], Path]]):
        for name, im in images.items():
            if isinstance(im, Path):
                im = imageio.imread(im)
            elif isinstance(im, Tensor):
                im = im.data
            print(f"{name}: {im.shape}")

else:
    def show_images(images: Dict[str, Union[Tensor, NDArray[Any], Path]]):
        v = napari.Viewer()
        for name, im in images.items():
            if isinstance(im, Path):
                im = imageio.imread(im)
            elif isinstance(im, Tensor):
                im = im.data
            print(f"napari viewer: adding {name}")
            v.add_image(im, name=name)

## Loading a model

We will use a model that predicts boundaries in images of plant cells [kaggle nucles segmentation challenge](https://www.kaggle.com/c/data-science-bowl-2018).
Find the model on bioimage.io here: ["affable-shark](https://bioimage.io/#/?id=10.5281%2Fzenodo.5764892)

In [ ]:
from bioimageio.spec import load_description

model = load_description("affable-shark/draft")

In [ ]:
# model alternative
from bioimageio.spec import load_description

model = load_description("emotional-cricket/draft")

Let's briefly checkout the validation summary created upon loading the description

In [ ]:
model.validation_summary.display()

In [ ]:
# the function 'test_model' from 'bioimageio.core.resource_tests' can be used to fully test the model,
# including running prediction for the test input(s) and checking that they agree with the test output(s)
# before using a model, it is recommended to check that it properly works with this function
# 'test_model' returns a dict with 'status'='passed'/'failed' and more detailed information
from bioimageio.core import test_model

test_summary = test_model(model)
test_summary.display()

## Prediction with the model

`bioimageio.core` implements functionality to run prediction with models desribed in the `bioimage.io` format.
This includes functions to run prediction on `numpy.ndarray`s/`xarray.DataArrays` as input and convenience functions to run predictions for images stored on disc.

In [ ]:
# Load the example image for this model, which is stored in numpy file format.
from bioimageio.spec.utils import load_array
from bioimageio.spec.model import v0_5

assert isinstance(model, v0_5.ModelDescr)
input_image = load_array(model.inputs[0].test_tensor)
print(f"array shape: {input_image.shape}")

In [ ]:
from bioimageio.core import Sample, Tensor

# Create a `Tensor` (light wrapper around an `xarray.DataArray`) from the test input image.
# `bioimageio.core.Tensors/xarray.DataArrays` are like numpy arrays, but they have annotated axes.
# The axes are used to validate that the axes of the input image match the axes expected by a model.
test_input_tensor = Tensor.from_numpy(input_image, dims=model.inputs[0].axes)

# print the axis annotations ('dims') and the shape of the input array
print(f"tensor shape: {test_input_tensor.tagged_shape}")

In [ ]:
# now we can create a sample --- a collection of tensors.
# In this case our model only has one input, but for models with multiple inputs a `Sample` includes a tensor for each input.
sample = Sample(members={"raw": test_input_tensor}, stat=None, id="sample-from-numpy")
sample

In [ ]:
# shortcut: helper function `create_sample_for_model` to create a sample for a given model directly

from bioimageio.core.digest_spec import create_sample_for_model
from bioimageio.spec.utils import download

input_paths = {ipt.id: download(ipt.test_tensor).path for ipt in model.inputs}
print(f"input paths: {input_paths}")
assert isinstance(model, v0_5.ModelDescr)
sample = create_sample_for_model(
    model=model, inputs=input_paths, sample_id="my_demo_sample"
)
sample

In [ ]:
# shortcut: get test input sample for a given model
from bioimageio.core.digest_spec import get_test_inputs

test_sample = get_test_inputs(model)
test_sample

In [ ]:
from bioimageio.core import create_prediction_pipeline

# Next, create a 'prediction_pipeline'. The prediction_pipeline is used to run prediction with a given model.
# This means it applies the preprocessing, runs inference with the model and applies the postprocessing.

# The 'devices' argument can be used to specify which device(s) to use for inference with the model.
# Hence it can be used to specify whether to use the cpu, a single gpu or multiple gpus (not implemented yet).
# By default (devices=None) a gpu will be used if available and otherwise the cpu will be used.
devices = None

# The 'weight_format' argument can be used to specify which weight format available in the model to use.
# By default (weight_format=None) the weight format with highest priority (as defined by bioimageio.core) will be used.
weight_format = None

prediction_pipeline = create_prediction_pipeline(
    model, devices=devices, weight_format=weight_format
)

In [ ]:
# Use the prediction pipeline to run prediction for the image we loaded before.
# The prediction pipeline returns a `Sample` object.
prediction: Sample = prediction_pipeline.predict_sample_without_blocking(sample)

# show the prediction result
show_images({**sample.members, **prediction.members})

there are convenience functions `predict` and `predict_many` that can be used to predict images without explicit creation of a `PredictionPipeline`... 

In [ ]:
from bioimageio.core import predict  # , predict_many

predict(model=model, inputs=sample)
# predict_many(model=model, inputs=[sample])